In [0]:
import pandas as pd

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()

In [0]:
df = pd.read_csv('job_ofer.csv', error_bad_lines=False, warn_bad_lines=False)
df.shape

(36109, 8)

In [0]:
df.head()

,title,company_name,address,description,seniority_level,employment_type,job_function,industries
0,Machine Learning Engineer,Intellipro Group Inc,"Palo Alto, CA, US","['About The Company', ""W*** is reshaping the f...",Entry level,Full-time,Engineering,Information Technology and Services
1,Deep Learning Applied Researcher - Chicago,Ethosia,"Chicago, IL, US","['תיאור המשרה', 'Deep learning for Computer Vi...",Associate,Full-time,Other,Information Technology and Services
2,Machine Learning Engineer,Motorola Solutions,"Chicago, IL, US","['Company Overview', 'At Motorola Solutions, w...",Entry level,Full-time,Engineering,Information Technology and Services
3,Machine Learning / Data Scientist,Proprius LLC,"San Francisco, CA, US",['Our client is a digital invention agency foc...,Entry level,Full-time,Engineering,Information Technology and Services
4,Cloud Architect,TCS,"Framingham, Massachusetts, United States","['Technical/Functional Skills', ' ', 'Good to ...",Mid-Senior level,Full-time,Engineering,Information Technology and Services


# **Word2Vec**

In [0]:
#  A list of individual tokens
corpus = df['title'].map( simple_preprocess )

In [0]:
model = Word2Vec(corpus, size=100, window=2, min_count=1)

In [0]:
model.wv.most_similar('machine')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('deep', 0.9475566148757935),
 ('edge', 0.8278223872184753),
 ('natural', 0.7996666431427002),
 ('bentonville', 0.793515682220459),
 ('nlu', 0.7873601317405701),
 ('big', 0.785819411277771),
 ('nlp', 0.7834916114807129),
 ('scientists', 0.7819380164146423),
 ('algorithm', 0.7786482572555542),
 ('personalization', 0.7744515538215637)]

#**Title + phrases**

In [0]:
title_corpus = df['title'].map( simple_preprocess )

title_bigram = Phraser( Phrases(title_corpus, min_count=1, threshold=1) )

In [0]:
#  Passing sentences through title_bigram
title_bigram[ simple_preprocess('Deep Learning Applied Researcher - Chicago') ]

['deep_learning', 'applied', 'researcher', 'chicago']

In [0]:
title_corpus_phrase = [ title_bigram[sent] for sent in title_corpus]
model = Word2Vec(title_corpus_phrase, size=100, window=2, min_count=1)

In [0]:
model.wv.most_similar('machine')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('container', 0.8177450895309448),
 ('iii_grades', 0.8123569488525391),
 ('askey', 0.8049790263175964),
 ('community_college', 0.8049756288528442),
 ('guide', 0.804436445236206),
 ('comptes', 0.8030059933662415),
 ('universal_parks', 0.801345944404602),
 ('ethics', 0.8012807369232178),
 ('store_merchandising', 0.8005654811859131),
 ('knee_reconstruction', 0.8002898693084717)]

In [0]:
#  Connected words by _ + single words
def prepare_corupus(corpus, bigram):
  for sent in corpus:
    yield bigram[sent] + sent

In [0]:
ext_corp = list(prepare_corupus(title_corpus, title_bigram))
title_model = Word2Vec( ext_corp, size=100, window=2, min_count=1 )

In [0]:
title_model.wv.most_similar('machine')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('deep', 0.9534536600112915),
 ('learning', 0.9381073713302612),
 ('machine_learning', 0.9323673844337463),
 ('computer_vision', 0.9206240177154541),
 ('learning_product', 0.9168796539306641),
 ('nlp_research', 0.9134105443954468),
 ('deep_learning', 0.9091349244117737),
 ('edge', 0.9082440137863159),
 ('hux', 0.9036274552345276),
 ('computer', 0.9027739763259888)]

# **Description**

In [0]:
#  train title_bigram
descr_corpus = df['description'].map( simple_preprocess )
descr_bigram = Phraser( Phrases(descr_corpus, min_count=1, threshold=1) )

In [0]:
ext_descr_corp = list(prepare_corupus(descr_corpus, descr_bigram))
descr_model = Word2Vec( ext_corp, size=100, window=2, min_count=1 )

In [0]:
descr_model.wv.most_similar('nlp')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('artificial_intelligence', 0.9928469061851501),
 ('ml', 0.9913045167922974),
 ('big_data', 0.9912072420120239),
 ('alpharetta', 0.9905535578727722),
 ('computer', 0.9893373847007751),
 ('deep_learning', 0.9875133037567139),
 ('computer_vision', 0.9845010042190552),
 ('deep', 0.9835388660430908),
 ('personalization', 0.9823084473609924),
 ('aws', 0.9788297414779663)]

In [0]:
title_model.wv.most_similar('nlp')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ml', 0.9911259412765503),
 ('computer_vision', 0.9899900555610657),
 ('applied_research', 0.9885576367378235),
 ('deep_learning', 0.9882297515869141),
 ('computer', 0.9873024225234985),
 ('applied', 0.9857988357543945),
 ('artificial_intelligence', 0.9856171011924744),
 ('geologist', 0.9837548732757568),
 ('alpharetta', 0.9828254580497742),
 ('big_data', 0.9825098514556885)]

In [0]:
descr_model.wv.most_similar('python')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('java', 0.9838465452194214),
 ('iib', 0.9810817837715149),
 ('android', 0.9771008491516113),
 ('backend', 0.9733642339706421),
 ('ios', 0.972675621509552),
 ('apply_now', 0.9717574119567871),
 ('senior_react', 0.969959020614624),
 ('bw', 0.9689472913742065),
 ('react', 0.9689252972602844),
 ('junior_developer', 0.9682441353797913)]